In [162]:
from langchain.llms import AzureOpenAI
import openai
from dotenv import load_dotenv
import os
from IPython.display import display, HTML, JSON
import tiktoken

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

OPENAI_DAVINCI_DEPLOYMENT_NAME = os.getenv("OPENAI_DAVINCI_DEPLOYMENT_NAME")
OPENAI_DAVINCI_MODEL_NAME = os.getenv("OPENAI_DAVINCI_MODEL_NAME")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY


### **Initialize the LLM model which is deployed in Azure with LangChain**

In [163]:

def init_llm(model=OPENAI_MODEL_NAME,
             deployment_name=OPENAI_DEPLOYMENT_NAME, 
             temperature=0,
             max_tokens=400,
             stop="<|im_end|>", 
             ):
    
    llm = AzureOpenAI(deployment_name=deployment_name,  
                  model=model,
                  temperature=temperature,) 
    return llm


### **Add personality to the model and ask questions**
We call directly the Azure OpenAI API with ***ChatCompletion*** API

In [164]:
#prepare prompt
messages=[{"role": "system", "content": "You are a HELPFUL assistant answering users trivia questions. Answer in a clear and concise manner."},
          { "role": "user", "content": "Good morning, how are you today?" }]
       

answer = openai.ChatCompletion.create(engine = OPENAI_DEPLOYMENT_NAME, 
                                      messages = messages,)
display (HTML("ChatCompletion (gpt-35-turbo) :" + answer.choices[0].message.content))


In [104]:

"""
If you try to run following code, you will get an error:
openai.error.InvalidRequestError: The chatCompletion operation does not work with the specified model, text-davinci-003. 
Please choose a different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.
"""
#Error!
#answer = openai.ChatCompletion.create(engine = "text-davinci-003",
#                                   messages = messages,)

'\nIf you try to run following code, you will get an error:\nopenai.error.InvalidRequestError: The chatCompletion operation does not work with the specified model, text-davinci-003. \nPlease choose a different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.\n'

In [165]:
#prepare prompt with another question:
messages=[{"role": "system", "content": "You are q HELPFUL assistant answering users trivia questions. Answer in clear and concise manner."},
          { "role": "user", "content": "What's string theory?" }]
       

answer = openai.ChatCompletion.create(engine = OPENAI_DEPLOYMENT_NAME, 
                                      messages = messages,)

#print("ChatCompletion (gpt-35-turbo) :" + answer.choices[0].message.content)

display(HTML(answer.choices[0].message.content))


In [106]:
#prepare prompt with another question:
messages=[{"role": "system", "content": "You are a HELPFUL assistant answering users trivia questions. Answer as for a FIVE YEARS old child."},
          { "role": "user", "content": "what's string theory?" }]
       

answer = openai.ChatCompletion.create(engine = OPENAI_DEPLOYMENT_NAME, 
                                      messages = messages,)

#print("ChatCompletion (gpt-35-turbo) :" + answer.choices[0].message.content)
display(HTML(answer.choices[0].message.content))



### **LangChain**

LangChain is a framework built around Large Language Models (LLMs).

The core idea of the library is that we can “chain” together different components to create more advanced use cases around LLMs.

In [166]:
llm=init_llm()
llm

AzureOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.completion.Completion'>, model_name='gpt-35-turbo', temperature=0.0, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, openai_proxy=None, batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False, allowed_special=set(), disallowed_special='all', deployment_name='gpt-35-turbo')

In [167]:
#model "gpt-35-turbo"  
#You can see that gpt-35-turbo has been trained in QnA conversational style.
answer=llm("Good morning, how are you?")
display (HTML("gpt-35-turbo: " + answer))

In [109]:
#model "text-davinci-003"
#text-davinci-003 is a more generic model, trained for the mode: text-in, text-out
llm=init_llm(OPENAI_DAVINCI_MODEL_NAME, OPENAI_DAVINCI_DEPLOYMENT_NAME)
answer=llm("Good morning, how are you?")
display(HTML("text-davinci-003: "+ answer))

In [110]:
llm=init_llm()
answer=llm("Create a Python function that takes a string argument and reverses it.")
display (HTML("gpt-35-turbo: " + answer))

In [14]:
llm=init_llm(OPENAI_DAVINCI_MODEL_NAME, OPENAI_DAVINCI_DEPLOYMENT_NAME)
answer=llm("Create a Python function that takes a string argument and reverses it.")
display (HTML("text-davinci-003: " + answer))

#### **Prompts with LangChain**

In [111]:
from langchain import PromptTemplate

#create template for prompt

template = """You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
            
            USER: {question}
            ASSISTANT:
            
            <|im_end|>
            """

llm=init_llm()
prompt_template = PromptTemplate(template=template, input_variables=["profession", "expertise", "question"])
answer = llm(prompt_template.format(profession="Financial Trading Consultant",  expertise="Risk Management",
                            question="How do you assess the risk tolerance of a new client?"))
display (HTML("gpt-35-turbo: " + answer))


In [123]:
#asking not related question
prompt_template = PromptTemplate(template=template, input_variables=["profession", "expertise", "question"])
answer = llm(prompt_template.format(profession="Financial Trading Consultant",  expertise="Risk Management",
                            question="What's the fastest car in the world?"))
display (HTML("gpt-35-turbo: " + answer))

##### ChatPromptTemplate is a template that is specifically designed for conversational use cases.

It infers the variables from the template, so no need to pass them in as arguments.

In [138]:
template = """You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
            
            USER: {question}
            ASSISTANT:
            
            <|im_end|>
            """

In [139]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template)


In [121]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['expertise', 'profession', 'question'], output_parser=None, partial_variables={}, template="You are a {profession} answering users questions. \n            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that the user is not a subject expert.\n            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.\n            \n            USER: {question}\n            ASSISTANT:\n            \n            <|im_end|>\n            ", template_format='f-string', validate_template=True)

In [140]:

answer = llm(prompt_template.format(profession="Financial Trading Consultant",  expertise="Risk Management",
                            question="What's the fastest car in the world?"))
display (HTML("gpt-35-turbo: " + answer))

#### LLM output parsers with LangChain
You can define the output schema and LangChain will parse the output for you.

The main idea is to define parsing instructions as a code rather than doing it in textual form.


In [125]:
from langchain.prompts import ChatPromptTemplate

In [126]:

customer_call = """

**Customer**: Hello, my name is John, and I'm a customer of Imaginal Bank.
**Clerk**: Hello, John! My name is Sara, and I'm a customer service representative at Imaginal Bank. How can I assist you today?
**Customer**: Hi, Sara. I'm interested in your bank's investment programs. 
              Can you tell me more about them, especially in terms of risk management?

**Clerk**: Absolutely, John. We have a few key programs I can highlight.

First, there's our 'Balanced Growth Fund'. It's a diversified mutual fund that invests in a mix of equities and bonds to provide both growth and income, reducing risk through diversification. 

We also have the 'Index Tracker ETF', which is designed to replicate the performance of a specific market index. By spreading investments across the entire index, it inherently reduces the risk associated with individual stocks.

Additionally, for those with a lower risk tolerance, we have the 'Secure Income Bond Fund', which focuses on government and high-quality corporate bonds. 

Our financial advisors are always available to guide you in choosing the right program based on your financial goals and risk tolerance.

**Customer**: I see. Could you elaborate on how the Balanced Growth Fund manages risk?

**Clerk**: Sure. The Balanced Growth Fund mitigates risk by diversifying investments across a wide range of assets. If one investment performs poorly, it's likely to be offset by other investments that are performing well. Furthermore, our portfolio managers actively manage the fund, adjusting holdings based on changing market conditions to manage risk and enhance returns.

**Customer**: Does the bank provide any tools to monitor my investments?

**Clerk**: Yes, John. We offer an online platform called 'Imaginal Investor Dashboard'. It provides real-time tracking of your investments, balance updates, and market trends. You can also set up alerts to be notified about significant changes in your portfolio.

**Customer**: That sounds quite comprehensive. How can I get started?

**Clerk**: You can schedule an appointment with one of our financial advisors. They'll walk you through your options, help you understand your risk tolerance, and guide you in choosing the right investment program. Would you like me to arrange that for you?

**Customer**: Yes, please. That would be helpful.

**Clerk**: Fantastic, John! Let's get that set up for you..."""


call_center_prompt_template = """

For the following text, extract the following information:
agent politeness: How polite is the agent? use the following values to descibe agent politenes: very polite, polite, neutral, impolite, very impolite.
agent knowledge: How knowledgeable is the agent? use the following values to descibe agent knowledge: very knowledgeable, knowledgeable, neutral, not knowledgeable, very not knowledgeable.
customer issue resolution: How well did the agent resolve the issue? use the following values to descibe issue resolution: very well, well, neutral, not well, very not well.
customer satisfaction: How satisfied is the customer? use the following values to descibe customer satisfaction: very satisfied, satisfied, neutral, dissatisfied, very dissatisfied.

Format the output as a json object with the following keys: agent_politeness, agent_knowledge, customer_issue_resolution, customer_satisfaction.

text: {text}
"""

In [127]:
prompt_template = ChatPromptTemplate.from_template(call_center_prompt_template)
#print (prompt_template)
llm=init_llm()
#Note that the type of the response is a string, it looks like a json object, but it's string
response = llm(prompt_template.format(text=customer_call))
print (response)


 
Output:

{
    "agent_politeness": "very polite",
    "agent_knowledge": "knowledgeable",
    "customer_issue_resolution": "well",
    "customer_satisfaction": "very satisfied"
}<|im_end|>


In [128]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [129]:

agent_politeness_schema = ResponseSchema(name="agent_politeness",description="How polite is the agent?")
aggent_knowledge_schema = ResponseSchema(name="agent_knowledge",description="How knowledgeable is the agent?")
customer_issue_resolution_schema = ResponseSchema(name="customer_issue_resolution",description="How well did the agent resolve the issue?")
customer_satisfaction_schema = ResponseSchema(name="customer_satisfaction",description="How satisfied is the customer?")
customer_service_schemas = [agent_politeness_schema,aggent_knowledge_schema,customer_issue_resolution_schema,customer_satisfaction_schema]

output_parser = StructuredOutputParser.from_response_schemas(customer_service_schemas)
format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"agent_politeness": string  // How polite is the agent?
	"agent_knowledge": string  // How knowledgeable is the agent?
	"customer_issue_resolution": string  // How well did the agent resolve the issue?
	"customer_satisfaction": string  // How satisfied is the customer?
}
```


In [130]:
call_center_prompt_template_with_output_parser = """

For the following text, extract the following information:
agent politeness: How polite is the agent? use the following values to descibe agent politenes: very polite, polite, neutral, impolite, very impolite.
agent knowledge: How knowledgeable is the agent? use the following values to descibe agent knowledge: very knowledgeable, knowledgeable, neutral, not knowledgeable, very not knowledgeable.
customer issue resolution: How well did the agent resolve the issue? use the following values to descibe issue resolution: very well, well, neutral, not well, very not well.
customer satisfaction: How satisfied is the customer? use the following values to descibe customer satisfaction: very satisfied, satisfied, neutral, dissatisfied, very dissatisfied.

Format the output as a json object with the following keys: agent_politeness, agent_knowledge, customer_issue_resolution, customer_satisfaction.

text: {text}

{format_instructions}
"""

In [131]:
prompt = ChatPromptTemplate.from_template(template=call_center_prompt_template_with_output_parser)
input = prompt.format(text=customer_call,format_instructions=format_instructions)


In [132]:
print (input)

Human: 

For the following text, extract the following information:
agent politeness: How polite is the agent? use the following values to descibe agent politenes: very polite, polite, neutral, impolite, very impolite.
agent knowledge: How knowledgeable is the agent? use the following values to descibe agent knowledge: very knowledgeable, knowledgeable, neutral, not knowledgeable, very not knowledgeable.
customer issue resolution: How well did the agent resolve the issue? use the following values to descibe issue resolution: very well, well, neutral, not well, very not well.
customer satisfaction: How satisfied is the customer? use the following values to descibe customer satisfaction: very satisfied, satisfied, neutral, dissatisfied, very dissatisfied.

Format the output as a json object with the following keys: agent_politeness, agent_knowledge, customer_issue_resolution, customer_satisfaction.

text: 

**Customer**: Hello, my name is John, and I'm a customer of Imaginal Bank.
**Cler

In [133]:
response = llm(input)

In [134]:
dict = output_parser.parse(response)
dict.get("agent_politeness")

'very polite'

#### Prompts management best practices
Reuse prompts as much as possible. This will help you to get more consistent results.

Treat your prompts as a code, keep it in a version control system. This will help you to track changes and to revert them if needed.

Use LangChain specific to use case prompt templates, e.g. ChatPromptTemplate for conversational flows.


#### **Using LLMChain** 

LLMChain is a simplest LangChain chain.

In [135]:
template = """You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
            
            USER: {question}
            ASSISTANT:
            
            <|im_end|>
            """

In [137]:
from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template(template=template)

#default llm is gpt-35-turbo
llm=init_llm()
# it's simplest langchain chain integrating llm and prompt
chain = LLMChain(llm=llm, prompt=prompt)

ans = chain.run(profession="Financial Trading Consultant",  expertise="Risk Management", 
          question= "How do you assess the risk tolerance of a new client, and how do you use this information in recommending trading strategies?")
display (HTML(ans))

###  **One-shot, Few-shot learning**

This technique could improve model performance by a lot. 
We can use the model to learn from a few examples and then use it to generate text. This is called few-shot learning. We can also use the model to learn from a single example and then use it to generate text. This is called one-shot learning.

In [19]:
template_few_shot = """You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that a user is not a subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
           
            USER: How do you assess the risk tolerance of a new client?
            ASSISTANT: I begin by having a comprehensive discussion with the client about their financial goals, investments horizon, and comfort level with different levels of risk.
            
            USER: Can you provide an example of a specific risk management strategy you'd recommended to a client in a volatile market situation?
            ASSISTANT: During the market volatility caused by the pandemic, I'd recommended that a client diversify their portfolio further to reduce risk exposure.
            
            USER: How do you handle the situation when a client wants to pursue a risky investment that goes beyond their risk tolerance?
            ASSISTANT: I would clearly communicate the potential risks associated with the investment and how it might not align with their established risk tolerance. 
            
            USER: {question}
            ASSISTANT:
            
            <|im_end|>
            """
           

In [20]:
prompt_few_shot = PromptTemplate(template=template_few_shot, input_variables=["profession", "expertise", "question"])
chain = LLMChain(llm=llm, prompt=prompt_few_shot)

res=chain.run(profession="Financial Trading Consultant",  expertise="Risk Management", 
          question= "How do you use technology or specific financial tools to assist in risk management for your clients?")
display (HTML(res))


##### Since we don't save a history of the conversation, the model will fail to answer questions that require context.

In [21]:
res = chain.run(profession="Financial Trading Consultant",  expertise="Risk Management", 
          question= "Which software do you use?")
display (HTML(res))

#### **LangChain Few-Shot learning**

In [22]:
from langchain import FewShotPromptTemplate
from langchain import PromptTemplate


# create few shot examples
examples = [
    {
        "query": "How do you assess the risk tolerance of a new client?",
        "answer": "I begin by having a comprehensive discussion with the client about their financial goals, investments horizon, and comfort level with different levels of risk.."
    }, 
    
    {
        "query": "Can you provide an example of a specific risk management strategy you've recommended to a client in a volatile market situation?",
        "answer": "During the market volatility caused by the pandemic, I recommended that a client diversify their portfolio further to reduce risk exposure."
    },
    {
        "query": "How do you handle the situation when a client wants to pursue a risky investment that goes beyond their risk tolerance?",
        "answer": "I would clearly communicate the potential risks associated with the investment and how it might not align with their established risk tolerance."
    },
    
]

# create a example template
example_template = """
User: {query}
Assistant: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix_template = """You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to {expertise} say: It's not clear or the question is not related to {expertise}.
examples: 
"""
# and the suffix our user input and output indicator
suffix_template = """
User: {query}
Assistant: """

prefix_prompt = PromptTemplate(input_variables=["profession", "expertise"], template=prefix_template)

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix = prefix_prompt.format(profession="Financial Trading Consultant",  expertise="Risk Management"),
    suffix=suffix_template,
    input_variables=["query"],
    example_separator="\n\n"
)


In [23]:
query="How do you use technology or specific financial tools to assist in risk management for your clients?"
display (HTML(few_shot_prompt_template.format(query=query)))

In [25]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)
ans = chain.run(few_shot_prompt_template.format(query=query))
display (HTML(ans))

### **Retain conversation history** 

##### The Large Language Models (LLMs) are stateless. This means that they don’t retain any information about the conversation history.
Each transaction is independent of the previous one. Chatbots keep in memory the conversation history and use it to generate the next response. This is why they are able to generate more coherent responses.

Previously we saw that a model fails to answer the question that requires context. We can solve this problem by retaining the conversation history. We can do this by using the LangChain ConversationBufferMemory.

In [156]:
template = """You are a {profession} answering users questions. 
            More specifically, you are an expert in {expertise}. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to {expertise} say: it's not clear or the question is not related to {expertise}.
            
            USER: {question}
            ASSISTANT:

            <|im_end|>
            """

In [155]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import ChatPromptTemplate

llm=init_llm()

#ConversationBufferMemory is a memory that stores the conversation history
memory = ConversationBufferMemory()
#try to change the verbose to True, to see more details
conversation = ConversationChain(llm=llm, memory=memory, verbose=False)


In [146]:
prompt = ChatPromptTemplate.from_template(template=template)

response = conversation.run (input=prompt.format(profession="Financial Trading Consultant",
                       expertise="Risk Management", 
                            question="How do you use technology or specific financial tools to assist in risk management for your clients?"))

display (HTML(ans))

In [149]:
response = conversation.run(input=prompt.format(profession="Financial Trading Consultant",
                       expertise="Risk Management", 
                            question="Which software do you use?"))

display (HTML(ans))

In [151]:
history = conversation.memory.chat_memory.messages
for msg in history:
    print ( f"{msg.type}: {msg.content}")
    

human: Human: You are a Financial Trading Consultant answering users questions. 
            More specifically, you are an expert in Risk Management. Answer in a clear and concise manner. Assume that the user is not a subject expert.
            If a question is not clear or not related to Risk Management say: it's not clear or the question is not related to Risk Management.
            
            USER: How do you use technology or specific financial tools to assist in risk management for your clients?
            ASSISTANT:
            
            <|im_end|>
            
ai:  Hello! I'm happy to help. There are many ways that technology and financial tools can be used to assist in risk management. One way is through the use of risk management software. This software can help identify potential risks and provide recommendations for how to mitigate them. Another way is through the use of financial models. These models can help predict future market trends and identify potential risks

In [152]:
#pay attention that LangChain added to the prompt: ```Current conversation:{history}```
print(conversation.prompt)

input_variables=['history', 'input'] output_parser=None partial_variables={} template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:' template_format='f-string' validate_template=True


In [154]:
print (memory.load_memory_variables({}))

{'history': 'Human: Human: You are a Financial Trading Consultant answering users questions. \n            More specifically, you are an expert in Risk Management. Answer in a clear and concise manner. Assume that the user is not a subject expert.\n            If a question is not clear or not related to Risk Management say: it\'s not clear or the question is not related to Risk Management.\n            \n            USER: How do you use technology or specific financial tools to assist in risk management for your clients?\n            ASSISTANT:\n            \n            <|im_end|>\n            \nAI:  Hello! I\'m happy to help. There are many ways that technology and financial tools can be used to assist in risk management. One way is through the use of risk management software. This software can help identify potential risks and provide recommendations for how to mitigate them. Another way is through the use of financial models. These models can help predict future market trends and 

In [153]:
ans = conversation.run (input = prompt.format(profession="Financial Trading Consultant",
                        expertise="Risk Management", 
                            question="List all questions I've asked you about Risk Management?"))
display (HTML(ans))

#### More conversation memory types.

Keeping full conversation history could be expensive and we can hit the Azure API limits. LangChain provides different types of conversation memory that can be used to keep the conversation history and mitigate the limits issues.

**1. ConversationBufferWindowMemory** - keeps the last N messages in the conversation history.

**2. ConversationTokenBufferMemory** - keeps the last N tokens in the conversation history.

**3. ConversationSummaryBufferMemory** - keeps summary of the conversation over time.

Additional Memory Type includes:

**4. Vector data memory** - stires the text in a vector DB and retrieves most semantically similar text.

**5. Entity memories** - use LLM which rememebers details about specific entities. 

Note: You can use multiple memories at the same time. 

Finally you can store the conevrsation history in a conventional database like SQL or NoSQL. 


In [158]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "hi"}, {"output": "hello"})
memory.save_context({"input": "What's your name"}, {"output": "My name is John"})

memory.load_memory_variables({})


{'history': "Human: What's your name\nAI: My name is John"}